# Amanzon SageMaker Ground Truth Demonstration for Video Frame Object Detection Labeling Job

1. [Introduction](#Introduction)
    1. [Cost and runtime](#cost-runtime)
    2. [Prerequisites](#prereq)
2. [Run a Ground Truth labeling job](#run-labeling-job)
    1. [Prepare the data](#Prepare-the-data)
    2. [Create a Video Frame Input Manifest File](#create-manifest)
    3. [Create the instruction template](#create-template)
    4. [Use a private team to test your task](#Create-a-private-team-to-test-your-task)
    5. [Define pre-built lambda functions for use in the labeling job](#lambda)
    6. [Submit the Ground Truth job request](#submit-req)
        1. [Verify your task using a private team](#verify-team)
    7. [Monitor job progress](#monitor)
    8. [View Task Results](#view-task)
3. [Clean Up - Optional](#cleanup)

## 1. Introduction <a class="anchor" id="Introduction"></a>

This sample notebook takes you through an end-to-end workflow to demonstrate the functionality of SageMaker Ground Truth Video Frame Object Detection.  You can use the video frame object Detection task type to have workers detect the objects in a sequence of video frames (images extracted from a video) using bounding boxes, polylines, polygons or keypoint annotation tools.

Before you begin, we highly recommend you start a Ground Truth labeling job through the AWS Console first to familiarize yourself with the workflow. The AWS Console offers less flexibility than the API, but is simple to use.

For more information, refer to Amazon SageMaker Developer Guide: [Video Frame Object Detection](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-video-object-detection.html).

#### Cost and runtime <a class="anchor" id="cost-runtime"></a>

1. For pricing, please refer to [Ground Truth pricing scheme](https://aws.amazon.com/sagemaker/groundtruth/pricing/). In order to reduce the cost, we will use Ground Truth's auto-labeling feature. Amazon SageMaker Ground Truth can use active learning to automate the labeling of your input data for certain built-in task types. Active learning is a machine learning technique that identifies data that should be labeled by your workers. In Ground Truth, this functionality is called automated data labeling. Automated data labeling helps to reduce the cost and time that it takes to label your dataset compared to using only humans.

#### Prerequisites <a class="anchor" id="prereq"></a>
To run this notebook, you can simply execute each cell one-by-one. To understand what's happening, you'll need:
* An S3 bucket you can write to -- please provide its name in the following cell. The bucket must be in the same region as this SageMaker Notebook instance. You can also change the `EXP_NAME` to any valid S3 prefix. All the files related to this experiment will be stored in that prefix of your bucket.
* Basic familiarity with [AWS S3](https://docs.aws.amazon.com/s3/index.html),
* Basic understanding of [AWS Sagemaker](https://aws.amazon.com/sagemaker/),
* Basic familiarity with [AWS Command Line Interface (CLI)](https://aws.amazon.com/cli/) -- set it up with credentials to access the AWS account you're running this notebook from. This should work out-of-the-box on SageMaker Jupyter Notebook instances.

This notebook is only tested on a SageMaker Studio Notebook & SageMaker Notebook Instances. The runtimes given are approximate, we used an `ml.t3.medium` instance with `Data Science` image. However, you can likely run it on a local instance by first executing the cell below on SageMaker, and then copying the `role` string to your local copy of the notebook.

NOTES: 
- This notebook will create/remove subdirectories in its working directory. We recommend to place this notebook in its own directory before running it. 

- Ground Truth requires all S3 buckets that contain labeling job input image data have a CORS policy attached. To learn more about this change, see CORS Permission Requirement https://docs.aws.amazon.com/sagemaker/latest/dg/sms-cors-update.html

In [ ]:
# cell 01

%load_ext autoreload
%autoreload 2

import os
import json
import time
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import boto3
import sagemaker
from urllib.parse import urlparse
import warnings

sess = sagemaker.Session()
BUCKET = sess.default_bucket() 

EXP_NAME = "label-video/video-frame-object-Detection"  # Any valid S3 prefix.

# VERIFY_USING_PRIVATE_WORKFORCE = True # private team leveraged for labelling job

In [ ]:
# cell 02

# Make sure the bucket is in the same region as this notebook.

role = sagemaker.get_execution_role()
region = boto3.session.Session().region_name

s3 = boto3.client("s3")
bucket_region = s3.head_bucket(Bucket=BUCKET)["ResponseMetadata"]["HTTPHeaders"][
    "x-amz-bucket-region"
]

assert (
    bucket_region == region
), f"You S3 bucket {BUCKET} and this notebook need to be in the same region."

## 2. Run a Ground Truth labeling job <a class="anchor" id="run-labeling-job"></a>


**This section should take about 30 min to complete.**

We will first run a labeling job. This involves several steps: collecting the video frames for labeling, specifying the possible label categories, creating instructions, and writing a labeling job specification.

### Prepare the data

For this demo, we have used a subset of 15 frames containing three main objects, mug, cofee and bottle water. This dataset contains images with a FPS of 25 and resultion of 512x512. For this demo, only a subset of data are downloaded in data directory.



We will copy these frames from data directory to our local `BUCKET`, and will create the corresponding *input manifest*. The input manifest is a formatted list of the S3 locations of the images we want Ground Truth to annotate. We will upload this manifest to our S3 `BUCKET`.


### Create a Video Frame Input Manifest File <a class="anchor" id="create-manifest"></a>
Ground Truth uses the input manifest file to identify the location of your input dataset when creating labeling tasks. For video frame object object detection labeling jobs, each line in the input manifest file identifies the location of a video frame sequence file. Each sequence file identifies the images included in a single sequence of video frames. For more information, click [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-video-manual-data-setup.html#sms-video-create-manifest)

In [ ]:
# cell 03

# upload frames to S3
# create manifest and manifest.json
# Manifest File: https://docs.aws.amazon.com/sagemaker/latest/dg/sms-input-data-input-manifest.html


CLASS_NAME = "Objects"
manifest_name = 'input.manifest'

total_frames = 0
frames = []
fr_no = 0
for i, filename in enumerate(sorted(os.listdir('./object_detection_data/'))):
    if filename.endswith(('jpg','jpeg','png')):
        total_frames += 1
        frames.append({"frame-no":fr_no,"frame":filename})
        s3.upload_file(f"./object_detection_data/{filename}", BUCKET, EXP_NAME + f"/{filename}")
        fr_no+=1
    
        
json_body = {
             "seq-no":1,
             f"prefix":f"s3://{BUCKET}/{EXP_NAME}/",
             "number-of-frames":total_frames,
             "frames":frames
            }

with open("./input.manifest.json", "w") as f:
    json.dump(json_body, f, separators=(',', ':'))
    
# upload the json file to s3
# s3.upload_file("class_labels.json", BUCKET, EXP_NAME + "/input/class_labels.json")

manifest = {"source-ref":f"s3://{BUCKET}/{EXP_NAME}/{manifest_name}.json"}
# Create a manifest (jsonline) file

with open(f"./{manifest_name}", "w") as outfile:
    json.dump(manifest, outfile, separators=(',', ':'))

In [ ]:
# cell 04

# Upload manifest and manifest.json files to S3
    
s3.upload_file("input.manifest", BUCKET, f"{EXP_NAME.split('/')[0]}" + "/input.manifest")
s3.upload_file("input.manifest.json", BUCKET, EXP_NAME + "/input.manifest.json")

### Create the Instruction Template <a class="anchor" id="create-template"></a>
 Specify labels and provide instructions for the workers

In [ ]:
# cell 05

# define the classes
json_body = {
  "labels": [
        {
          "label": "mug"
        },
        {
          "label": "coffee"
        },
        {
          "label": "bottle water"
        },
      ],
      "instructions": {
        "shortInstruction": "<p>Please draw bounding box for each object in each frame</p>",
        "fullInstruction": "<ul><li>Use the navigation bar in the bottom-left corner to see all video frames included in this task. Label each frame. If an object appears in more than one frame, use the same Label ID for that object in each frame.&nbsp;</li><li>Use the predict next icon <img src=\"https://a.b.cdn.console.awsstatic.com/a/v1/6JV62IARUU3XPV5JLTJ4EADRGVBU4S3TV5VOA5HGQTNOAD54IKUQ/src/images/PredictNext.svg\" style=\"max-width:100%\" alt=\"Predict next\">, or the shortcut command <strong>P</strong>, to have the user interface automatically infer the location of bounding boxes in subsequent frames for objects once you’ve placed a single bounding box around an object. Adjust the location and dimensions these inferred boxes as needed.&nbsp;</li><li>After you add a bounding box, adjust the box to fit tightly around the boundaries of an object or a person.</li><li>Once you add a bounding box, select the associated label in the <strong>Labels</strong> menu to add label attributes, if applicable.&nbsp;</li><li>Use the <strong>Shortcuts</strong> menu to see keyboard shortcuts that you can use to label objects faster.</li><li>Use this <a href=\"https://docs.aws.amazon.com/sagemaker/latest/dg/sms-video-object-detection.html#sms-video-ot-worker-ui\" rel=\"noopener noreferrer\" target=\"_blank\" style=\"color: rgb(68, 185, 214);\">resource</a> to learn about worker portal navigation, tools available to complete your task, icons, and view options.</li></ul>"
          }
    }

# upload the json to s3
with open("class_labels.json", "w") as f:
    json.dump(json_body, f)

s3.upload_file("class_labels.json", BUCKET, EXP_NAME + "/class_labels.json")

## 3.4 Use a private team to test your task


Refer to Prerequisites to setup private workforce team. 

In [ ]:
# cell 06

# private workforce team

private_workteam_arn = "<Your-Work-Team-ARN>"

assert (
    private_workteam_arn != "<Your-Work-Team-ARN>"
), "Please enter your private workforce team, private_workteam_arn. You can find it on Amazon SageMaker console > Ground Truth > Labeling workforces > Private Teams"


WORKTEAM_ARN = private_workteam_arn
print("WORKTEAM_ARN : {}".format(WORKTEAM_ARN))

## Define pre-built lambda functions for use in the labeling job <a class="anchor" id="lambda"></a>

In [ ]:
# cell 07

# Specify ARNs for resources needed to run an object detection job.
ac_arn_map = {
    "us-west-2"     : "081040173940",
    "us-east-1"     : "432418664414",
    "us-east-2"     : "266458841044",
    "eu-west-1"     : "568282634449",
    "ap-northeast-1": "477331159723",
}

# PreHumanTaskLambdaArn for VideoObjectDetection
prehuman_arn = f"arn:aws:lambda:{region}:{ac_arn_map[region]}:function:PRE-VideoObjectDetection"

# AnnotationConsolidationConfig for VideoObjectDetection
acs_arn = f"arn:aws:lambda:{region}:{ac_arn_map[region]}:function:ACS-VideoObjectDetection" 

# auto-labelling job
# https://docs.aws.amazon.com/sagemaker/latest/dg/sms-automated-labeling.html
labeling_algorithm_specification_arn = f"arn:aws:sagemaker:{region}:027400017018:labeling-job-algorithm-specification/object-detection"


## Submit the Ground Truth job request <a class="anchor" id="submit-req"></a>
The API starts a Ground Truth job by submitting a request. The request contains the 
full configuration of the annotation task, and allows you to modify the fine details of
the job that are fixed to default values when you use the AWS Console. The parameters that make up the request are described in more detail in the [SageMaker Ground Truth documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateLabelingJob.html).

After you submit the request, you should be able to see the job in your AWS Console, at `Amazon SageMaker > Labeling Jobs`.
You can track the progress of the job there. This job will take several hours to complete. If your job
is larger (say 10,000 review text), the speed and cost benefit of auto-labeling should be larger.

### Verify your task using a private team [OPTIONAL] <a class="anchor" id="verify-team"></a>
Run the next two cells. This will define the task and submit it to the private workforce (to you).
3. After a few minutes, you should be able to see your task in your private workforce interface.
Please verify that the task appears as you want it to appear.
4. If everything is in order, change `VERIFY_USING_PRIVATE_WORKFORCE` to `False` and rerun the cell below to start the real annotation task!

In [ ]:
# cell 08

# task definitions
task_description = f'Detecting objects in frames. Please draw a box around each object. Thank you!'
task_keywords = ['Video Frame Object Detection']
task_title = 'Video object Detection'
job_name = "video-frame-object-Detection-" + str(int(time.time()))
no_human_per_object = 1 # number of  workers required to label each text.
task_time_limit = 28800 # worker has to complete a task within 8 hours
task_availability_lifetime = 21600  # 6 hours to complete all pending tasks by human worker(s)
max_concurrent_task_count = 100 #maximum number of data objects that can be labeled by human workers at the same time
USE_AUTO_LABELING = False

In [ ]:
# cell 09

human_task_config={
        'PreHumanTaskLambdaArn': prehuman_arn,
        'TaskKeywords': task_keywords,
        'TaskTitle': task_title,
        'TaskDescription': task_description,
        'NumberOfHumanWorkersPerDataObject': no_human_per_object,  
        'TaskTimeLimitInSeconds': task_time_limit,  
        'TaskAvailabilityLifetimeInSeconds': task_availability_lifetime,
        'MaxConcurrentTaskCount': max_concurrent_task_count,
        'AnnotationConsolidationConfig': {
        'AnnotationConsolidationLambdaArn': acs_arn,
        },
        'UiConfig': {
            'HumanTaskUiArn': f"arn:aws:sagemaker:{region}:394669845002:human-task-ui/VideoObjectDetection",
        },
    }
    
human_task_config["WorkteamArn"] = private_workteam_arn

    
ground_truth_request = {
    "InputConfig":{
        'DataSource': {
            'S3DataSource': {
                'ManifestS3Uri': f"s3://{BUCKET}/{EXP_NAME.split('/')[0]}/{manifest_name}",
            }
        },
        'DataAttributes': {
            'ContentClassifiers': [
                'FreeOfPersonallyIdentifiableInformation','FreeOfAdultContent',
            ]
        }
    },
    "OutputConfig":{
        'S3OutputPath': f"s3://{BUCKET}/{EXP_NAME}/output/",
    },
    
    "HumanTaskConfig": human_task_config,
    "LabelingJobName": job_name,
    "RoleArn": role,
    "LabelAttributeName": "category-ref",
    "LabelCategoryConfigS3Uri": f"s3://{BUCKET}/{EXP_NAME}/class_labels.json",

}

if USE_AUTO_LABELING:
    ground_truth_request["LabelingJobAlgorithmsConfig"] = {
        "LabelingJobAlgorithmSpecificationArn": labeling_algorithm_specification_arn
    }
sagemaker_client = boto3.client("sagemaker")
sagemaker_client.create_labeling_job(**ground_truth_request)

## Monitor job progress <a class="anchor" id="monitor"></a>
You can monitor the job's progress through AWS Console. In this notebook, we will use Ground Truth output files and Cloud Watch logs in order to monitor the progress. You can re-evaluate the next two cells repeatedly. It sends a `describe_labelging_job` request which should tell you whether the job is completed or not. If it is, then 'LabelingJobStatus' will be 'Completed'.

In [ ]:
# cell 10

# re-evaluate repeatedly. It sends a `describe_labelging_job` request which should tell you whether the job is completed or not. If it is, then 'LabelingJobStatus' will be 'Completed'.
while sagemaker_client.describe_labeling_job(LabelingJobName=job_name)['LabelingJobStatus'] == 'InProgress':
    job_status = sagemaker_client.describe_labeling_job(LabelingJobName=job_name)['LabelingJobStatus']
    print('Labelling job : {}, status : {}'.format(job_name, job_status))
    time.sleep(45)
print('Labelling job : {}, status : {}'.format(job_name, sagemaker_client.describe_labeling_job(LabelingJobName=job_name)['LabelingJobStatus']))

## View Task Results  <a class="anchor" id="view-task"></a>
Once work is completed, Amazon SageMaker GroundTruth stores results in your S3 bucket and sends a Cloudwatch event. Your results should be available in the S3 OUTPUT_PATH when all work is completed.

In [ ]:
# cell 11

# ouput path
S3_OUTPUT = boto3.client('sagemaker').describe_labeling_job(LabelingJobName=job_name)['OutputConfig']['S3OutputPath'] + job_name
print('S3 OUPUT_PATH : {}'.format(S3_OUTPUT))

# Download human annotation data.
!aws s3 cp {S3_OUTPUT + '/manifests/output/output.manifest'} "./output/" #--recursive --quiet

In [ ]:
# cell 12

data=[]
with open('./output/output.manifest') as f:
    for line in f:
        json_data = json.loads(line)
        data.append(json_data)
        
print(data)

## Clean Up [OPTIONAL] <a class="anchor" id="cleanup"></a>
Finally, let's clean up and delete this endpoint.

In [ ]:
# cell 13

if sagemaker_client.describe_labeling_job(LabelingJobName=job_name)['LabelingJobStatus'] == 'InProgress':
    sagemaker_client.stop_labeling_job(LabelingJobName=job_name)

## The End!